## Medial Axis


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from udacity_helpers.grid import create_grid
from skimage.morphology import medial_axis
from skimage.util import invert
from planning import a_star
%matplotlib inline 

ImportError: cannot import name '_validate_lengths'

In [ ]:
plt.rcParams['figure.figsize'] = 12, 12

In [ ]:
# This is the same obstacle data from the previous lesson.
filename = 'colliders.csv'
data = np.loadtxt(filename, delimiter=',', dtype='Float64', skiprows=2)
print(data)

Starting and goal positions in *(north, east)*.

In [ ]:
start_ne = (25,  100)
goal_ne = (650, 500)

In [ ]:
# Static drone altitude (meters)
drone_altitude = 7
safety_distance = 5

In [ ]:
grid = create_grid(data, drone_altitude, safety_distance)
skeleton = medial_axis(invert(grid))


Plot the edges on top of the grid along with start and goal locations.

In [ ]:
# equivalent to
# plt.imshow(np.flip(grid, 0))

plt.imshow(grid, cmap='Greys', origin='lower')
plt.imshow(skeleton, cmap='Greys', origin='lower', alpha=0.7)
    
plt.plot(start_ne[1], start_ne[0], 'rx')
plt.plot(goal_ne[1], goal_ne[0], 'rx')

plt.xlabel('EAST')
plt.ylabel('NORTH')
plt.show()

In [ ]:
# The start and goal location defined above
# will not necessarily be on the skeleton so we
# must first identify the nearest cell on the 
# skeleton to start and goal

def find_start_goal(skel, start, goal):
    # Find the nearest point on the skeleton to the
    # start and goal positions
    
    # Find all non-zero elements of the skeleton array
    nz = np.transpose(np.nonzero(skel))
    # Calculate the distance of all non-zero elements to the start
    start_dist = np.linalg.norm(nz - np.array(start), axis=1)
    # Find the index of the minimum distance element and get the
    # skeleton coordinates
    near_start = nz[start_dist.argmin()]

    # Calculate the distance of all non-zero elements to the goal
    goal_dist = np.linalg.norm(nz - np.array(goal), axis=1)
    # Find the index of the minimum distance element and get the
    # skeleton coordinates
    near_goal = nz[goal_dist.argmin()]
    
    return tuple(near_start), tuple(near_goal)

skel_start, skel_goal = find_start_goal(skeleton, start_ne, goal_ne)

print(start_ne, goal_ne)
print(skel_start, skel_goal)


In [ ]:
def heuristic_func(position, goal_position):
    # Use euclidean distance as the heuristic for now
    return np.sqrt((goal_position[0] - position[0])**2 +
                   (goal_position[1] - position[1])**2)


### TODO: Run A* on the skeleton
see [planning.py](/edit/planning.py) for a reminder on how to run the imported A* implementation (or rewrite it!)

In [ ]:
# The skeleton array has 'True' for all pixels corresponding to the
# skeleton lines, but true in our map means that an obstacle exists
# in that cell. We need to invert the skeleton map so everywhere
# looks like an obstacle except the skeleton lines.
path, cost = a_star(invert(skeleton).astype(np.int), 
                    heuristic_func, skel_start, skel_goal)



In [ ]:
# Compare to regular A* on the grid
path2, cost2 = a_star(grid, heuristic_func, start_ne, goal_ne)


In [ ]:
plt.imshow(grid, cmap='Greys', origin='lower')
plt.imshow(skeleton, cmap='Greys', origin='lower', alpha=0.7)
# For the purposes of the visual the east coordinate lay along
# the x-axis and the north coordinates long the y-axis.
plt.plot(start_ne[1], start_ne[0], 'x')
# Uncomment the following as needed
plt.plot(goal_ne[1], goal_ne[0], 'x')

pp = np.array(path)
plt.plot(pp[:, 1], pp[:, 0], 'g')
pp2 = np.array(path2)
plt.plot(pp2[:, 1], pp2[:, 0], 'r')

plt.xlabel('EAST')
plt.ylabel('NORTH')
plt.show()

[solution](/notebooks/Medial-Axis-Solution.ipynb)